In [15]:
# Import stuff
from bs4 import BeautifulSoup
from lxml import html
import requests
import pandas as pd
import numpy as np
import re

In [16]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
# Get the page using requests
page = "https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik/top/plus/0/galerie/0?saison_id=2000"
pageTree = requests.get(page, headers=headers)
# Get the page soup using html parser
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

In [17]:
# Find the relevant selectors like this
Players = pageSoup.find_all("a", {"class": "spielprofil_tooltip"})

In [18]:
Players[0].text

'Luís Figo'

In [19]:
Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})

Values[0].text

'£54.00m'

In [20]:
PlayersList = []
ValuesList = []

for i in range(0,25):
    PlayersList.append(Players[i].text)
    ValuesList.append(Values[i].text)
    
df = pd.DataFrame({"Players":PlayersList,"Values":ValuesList})

df

,Players,Values
0,Luís Figo,£54.00m
1,Hernán Crespo,£51.13m
2,Marc Overmars,£36.00m
3,Gabriel Batistuta,£32.54m
4,Nicolas Anelka,£31.05m
5,Rio Ferdinand,£23.40m
6,Flávio Conceição,£22.50m
7,Savo Milosevic,£22.50m
8,David Trézéguet,£20.92m
9,Matías Almeyda,£20.70m


In [21]:
page = requests.get('https://www.premierleague.com/clubs')
tree = html.fromstring(page.content)

In [22]:
#Using the page's CSS classes, extract all links pointing to a team
linkLocation = tree.cssselect('.indexItem')

#Create an empty list for us to send each team's link to
teamLinks = []

#For each link...
for i in range(0,20):
    
    #...Find the page the link is going to...
    temp = linkLocation[i].attrib['href']
    
    #...Add the link to the website domain...
    temp = "http://www.premierleague.com/" + temp
    
    #...Change the link text so that it points to the squad list, not the page overview...
    temp = temp.replace("overview", "squad")
    
    #...Add the finished link to our teamLinks list...
    teamLinks.append(temp)
teamLinks

['http://www.premierleague.com//clubs/1/Arsenal/squad',
 'http://www.premierleague.com//clubs/2/Aston-Villa/squad',
 'http://www.premierleague.com//clubs/130/Brentford/squad',
 'http://www.premierleague.com//clubs/131/Brighton-and-Hove-Albion/squad',
 'http://www.premierleague.com//clubs/43/Burnley/squad',
 'http://www.premierleague.com//clubs/4/Chelsea/squad',
 'http://www.premierleague.com//clubs/6/Crystal-Palace/squad',
 'http://www.premierleague.com//clubs/7/Everton/squad',
 'http://www.premierleague.com//clubs/9/Leeds-United/squad',
 'http://www.premierleague.com//clubs/26/Leicester-City/squad',
 'http://www.premierleague.com//clubs/10/Liverpool/squad',
 'http://www.premierleague.com//clubs/11/Manchester-City/squad',
 'http://www.premierleague.com//clubs/12/Manchester-United/squad',
 'http://www.premierleague.com//clubs/23/Newcastle-United/squad',
 'http://www.premierleague.com//clubs/14/Norwich-City/squad',
 'http://www.premierleague.com//clubs/20/Southampton/squad',
 'http://www

In [23]:
#Create empty lists for player links
playerLink1 = []
playerLink2 = []

#For each team link page...
for i in range(len(teamLinks)):
    
    #...Download the team page and process the html code...
    squadPage = requests.get(teamLinks[i])
    squadTree = html.fromstring(squadPage.content)
    
    #...Extract the player links...
    playerLocation = squadTree.cssselect('.playerOverviewCard')

    #...For each player link within the team page...
    for i in range(len(playerLocation)):
        #...Save the link, complete with domain...
        playerLink1.append("http://www.premierleague.com/" + playerLocation[i].attrib['href'])
        
        #...For the second link, change the page from player overview to stats
        playerLink2.append(playerLink1[-1].replace("overview", "stats"))
for i in range(len(playerLink1)):
    print(playerLink1[i],playerLink2[i])

http://www.premierleague.com//players/4985/Bernd-Leno/overview http://www.premierleague.com//players/4985/Bernd-Leno/stats
http://www.premierleague.com//players/49716/Rúnar-Alex-Rúnarsson/overview http://www.premierleague.com//players/49716/Rúnar-Alex-Rúnarsson/stats
http://www.premierleague.com//players/23650/Arthur-Okonkwo/overview http://www.premierleague.com//players/23650/Arthur-Okonkwo/stats
http://www.premierleague.com//players/51917/Karl-Hein/overview http://www.premierleague.com//players/51917/Karl-Hein/stats
http://www.premierleague.com//players/4474/Héctor-Bellerín/overview http://www.premierleague.com//players/4474/Héctor-Bellerín/stats
http://www.premierleague.com//players/12185/Kieran-Tierney/overview http://www.premierleague.com//players/12185/Kieran-Tierney/stats
http://www.premierleague.com//players/50234/Gabriel-Magalhães/overview http://www.premierleague.com//players/50234/Gabriel-Magalhães/stats
http://www.premierleague.com//players/11575/Rob-Holding/overview http:/

In [25]:
#Create lists for each variable
Name = []
Team = []
Age = []
Apps = []
HeightCM = []
WeightKG = []


#Populate lists with each player

#For each player...
for i in range(len(playerLink1)):

    #...download and process the two pages collected earlier...
    playerPage1 = requests.get(playerLink1[i])
    playerTree1 = html.fromstring(playerPage1.content)
    playerPage2 = requests.get(playerLink2[i])
    playerTree2 = html.fromstring(playerPage2.content)

    #...find the relevant datapoint for each player, starting with name...
    try:
        tempName = str(playerTree1.cssselect('div.name')[0].text_content())
    except IndexError:
        tempTeam = str("BLANK")
    
    #...and team, but if there isn't a team, return "BLANK"...
    try:
        tempTeam = str(playerTree1.cssselect('.table:nth-child(1) .long')[0].text_content())
    except IndexError:
        tempTeam = str("BLANK")
    
    #...and age, but if this isn't there, leave a blank 'no number' number...
    
    try:  
        tempAge = str(playerTree1.cssselect('.pdcol2 li:nth-child(1) .info')[0].text_content())
    except IndexError:
        tempAge = float('NaN')

    #...and appearances. This is a bit of a mess on the page, so tidy it first...
    try:
        tempApps = playerTree2.cssselect('.statappearances')[0].text_content()
        tempApps = int(re.search(r'\d+', tempApps).group())
    except IndexError:
        tempApps = float('NaN')

    #...and height. Needs tidying again...
    try:
        tempHeight = playerTree1.cssselect('.pdcol3 li:nth-child(1) .info')[0].text_content()
        tempHeight = int(re.search(r'\d+', tempHeight).group())
    except IndexError:
        tempHeight = float('NaN')

    #...and weight. Same with tidying and returning blanks if it isn't there
    try:
        tempWeight = playerTree1.cssselect('.pdcol3 li+ li .info')[0].text_content()
        tempWeight = int(re.search(r'\d+', tempWeight).group())
    except IndexError:
        tempWeight = float('NaN')

#     print(tempName,tempApps)
    #Now that we have a player's full details - add them all to the lists
    Name.append(tempName)
    Team.append(tempTeam)
    Age.append(tempAge)
    Apps.append(tempApps)
    HeightCM.append(tempHeight)
    WeightKG.append(tempWeight)

In [26]:
def FindAge(x):
    pass
for i in range(len(Age)):
#     print(str(Age[i]).replace('\n',''))
    Age[i] = str(Age[i])
    m = re.search(r"\(([A-Za-z0-9_]+)\)", Age[i])
    try:
        Age[i] = m.group(1)
    except AttributeError:
        Age[i] = float('NaN')

In [27]:
df = pd.DataFrame(
    {'Name':Name,
     'Team':Team,
     'Age':Age,
     'Apps':Apps,
     'HeightCM':HeightCM,
     'WeightKG':WeightKG})
df.head()

,Name,Team,Age,Apps,HeightCM,WeightKG
0,Bernd Leno,Arsenal,29,98,190.0,83.0
1,RÃºnar Alex RÃºnarsson,Arsenal,26,1,186.0,76.0
2,Arthur Okonkwo,Arsenal,19,0,198.0,NaN
3,Karl Hein,Arsenal,19,0,193.0,NaN
4,HÃ©ctor BellerÃ­n,Arsenal,26,183,178.0,74.0


In [ ]:
df

In [ ]:
#Show me Karius' height:

df[df['Name']=="Loris Karius"]["HeightCM"]


In [ ]:
df.to_csv("EPLData.csv")

In [ ]:
2+2